# UTE and ZTE Trajectories

This notebook demonstrates the generation of Ultra-Short Echo time (UTE) and Zero Echo Time (ZTE) style trajectories. These typically involve ramp sampling from the k-space center outwards.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D # For 3D plotting
from trajgen import KSpaceTrajectoryGenerator, Trajectory

# Ensure plots appear inline in the notebook
%matplotlib inline

## 1. UTE 3D Radial Trajectory

This demonstrates a 3D radial trajectory with ramp sampling enabled, characteristic of UTE sequences. Each spoke starts from the k-space center and goes outwards.

In [ ]:
# Instantiate generator for a UTE 3D radial trajectory
gen_ute_radial = KSpaceTrajectoryGenerator(
    traj_type='radial3d',
    dim=3,
    fov=0.200,           # meters
    resolution=0.004,    # meters
    n_interleaves=128,   # Number of radial spokes
    use_golden_angle=True,
    ute_ramp_sampling=True # Key flag for UTE behavior
)

# Generate trajectory waveforms
kx_urad, ky_urad, kz_urad, gx_urad, gy_urad, gz_urad, t_urad = gen_ute_radial.generate()

# Combine all interleaves (spokes)
kspace_ute_radial_3d = np.stack([kx_urad.ravel(), ky_urad.ravel(), kz_urad.ravel()])
gradients_ute_radial_3d = np.stack([gx_urad.ravel(), gy_urad.ravel(), gz_urad.ravel()])

# Create Trajectory object
traj_ute_radial = Trajectory(
    name='UTE 3D Radial Example',
    kspace_points_rad_per_m=kspace_ute_radial_3d,
    gradient_waveforms_Tm=gradients_ute_radial_3d,
    dt_seconds=gen_ute_radial.dt,
    metadata={'gamma_Hz_per_T': gen_ute_radial.gamma, 'generator_params': gen_ute_radial.__dict__}
)

# Display trajectory summary
traj_ute_radial.summary()

In [ ]:
# Plot a subset of the UTE 3D Radial k-space trajectory
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
step = max(1, kspace_ute_radial_3d.shape[1] // 1000) # Aim for around 1000 points
ax.plot(kspace_ute_radial_3d[0, ::step], kspace_ute_radial_3d[1, ::step], kspace_ute_radial_3d[2, ::step], '.', markersize=1)
ax.set_title('UTE 3D Radial K-Space Trajectory (Half-Spokes)')
ax.set_xlabel('Kx (rad/m)')
ax.set_ylabel('Ky (rad/m)')
ax.set_zlabel('Kz (rad/m)')
plt.show()

## 2. ZTE Trajectory

The 'zte' trajectory type is an alias for 'radial3d'. To achieve ZTE's characteristic center-out ramp sampling, `ute_ramp_sampling` must be set to `True`.

In [ ]:
# Instantiate generator for a ZTE trajectory
gen_zte = KSpaceTrajectoryGenerator(
    traj_type='zte',      # Using the 'zte' alias
    dim=3,
    fov=0.200,            # meters
    resolution=0.004,     # meters
    n_interleaves=128,    # Number of radial spokes
    use_golden_angle=True,
    ute_ramp_sampling=True # CRITICAL: This enables the center-out ramp sampling for ZTE
)

# Generate trajectory waveforms
kx_zte, ky_zte, kz_zte, gx_zte, gy_zte, gz_zte, t_zte = gen_zte.generate()

# Combine all interleaves (spokes)
kspace_zte_3d = np.stack([kx_zte.ravel(), ky_zte.ravel(), kz_zte.ravel()])
gradients_zte_3d = np.stack([gx_zte.ravel(), gy_zte.ravel(), gz_zte.ravel()])

# Create Trajectory object
traj_zte = Trajectory(
    name='ZTE Trajectory Example',
    kspace_points_rad_per_m=kspace_zte_3d,
    gradient_waveforms_Tm=gradients_zte_3d,
    dt_seconds=gen_zte.dt,
    metadata={'gamma_Hz_per_T': gen_zte.gamma, 'generator_params': gen_zte.__dict__}
)

# Display trajectory summary
traj_zte.summary()

In [ ]:
# Plot a subset of the ZTE k-space trajectory
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
step = max(1, kspace_zte_3d.shape[1] // 1000) # Aim for around 1000 points
ax.plot(kspace_zte_3d[0, ::step], kspace_zte_3d[1, ::step], kspace_zte_3d[2, ::step], '.', markersize=1)
ax.set_title('ZTE K-Space Trajectory (Center-Out Ramped Radial)')
ax.set_xlabel('Kx (rad/m)')
ax.set_ylabel('Ky (rad/m)')
ax.set_zlabel('Kz (rad/m)')
plt.show()

## 3. UTE Cones Trajectory

In [ ]:
# Instantiate generator for a UTE Cones trajectory
gen_ute_cones = KSpaceTrajectoryGenerator(
    traj_type='cones',
    dim=3,
    fov=0.200,            # meters
    resolution=0.005,     # meters
    n_interleaves=32,     # Number of cones
    use_golden_angle=False, # For cones, n_interleaves directly determines phi angles usually
    ute_ramp_sampling=True, # Enable ramp sampling for UTE behavior
    vd_method='power',    # Example: use power law for variable density along cone axis
    vd_alpha=0.75
)

# Generate trajectory waveforms
kx_ucones, ky_ucones, kz_ucones, gx_ucones, gy_ucones, gz_ucones, t_ucones = gen_ute_cones.generate()

# Combine all interleaves (cones)
kspace_ute_cones_3d = np.stack([kx_ucones.ravel(), ky_ucones.ravel(), kz_ucones.ravel()])
gradients_ute_cones_3d = np.stack([gx_ucones.ravel(), gy_ucones.ravel(), gz_ucones.ravel()])

# Create Trajectory object
traj_ute_cones = Trajectory(
    name='UTE Cones Example',
    kspace_points_rad_per_m=kspace_ute_cones_3d,
    gradient_waveforms_Tm=gradients_ute_cones_3d,
    dt_seconds=gen_ute_cones.dt,
    metadata={'gamma_Hz_per_T': gen_ute_cones.gamma, 'generator_params': gen_ute_cones.__dict__}
)

# Display trajectory summary
traj_ute_cones.summary()

In [ ]:
# Plot a subset of the UTE Cones k-space trajectory
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
step = max(1, kspace_ute_cones_3d.shape[1] // 1000)
ax.plot(kspace_ute_cones_3d[0, ::step], kspace_ute_cones_3d[1, ::step], kspace_ute_cones_3d[2, ::step], '.', markersize=1)
ax.set_title('UTE Cones K-Space Trajectory (Ramp Sampled)')
ax.set_xlabel('Kx (rad/m)')
ax.set_ylabel('Ky (rad/m)')
ax.set_zlabel('Kz (rad/m)')
ax.view_init(elev=30, azim=30)
plt.show()

This notebook illustrated UTE-style radial and cone trajectories, as well as the ZTE trajectory (functionally similar to UTE radial). The key `ute_ramp_sampling=True` flag ensures that these trajectories sample from the k-space center outwards.